In [7]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import joblib
import os
import warnings

def predict_next_6_months(commodity, market, df):
    """
    Predict next 6 months of Modal Price using pretrained LSTM model.

    Args:
        commodity (str): Commodity name (e.g., 'Apple')
        market (str): Market name (e.g., 'Mechua')
        df (pd.DataFrame): Full historical dataset

    Returns:
        dict: {date (str): forecasted price (float)}
    """
    try:
        model_path = f"lstm_models/lstm_{commodity}_{market}.h5"
        scaler_path = f"scalers/scaler_{commodity}_{market}.pkl"

        if not os.path.exists(model_path) or not os.path.exists(scaler_path):
            raise FileNotFoundError(f"Model or scaler for {commodity}-{market} not found.")

        # Load model and scaler
        model = load_model(model_path, compile=False)
        scaler = joblib.load(scaler_path)

        # Preprocess
        data = df[(df['Commodity'] == commodity) & (df['Market'] == market)][['Arrival_Date', 'Modal_Price']].copy()
        data['Arrival_Date'] = pd.to_datetime(data['Arrival_Date'], dayfirst=True, errors='coerce')
        data.dropna(subset=['Arrival_Date', 'Modal_Price'], inplace=True)
        data.set_index('Arrival_Date', inplace=True)
        data.index = pd.DatetimeIndex(data.index)
        monthly = data.resample("ME").mean().dropna()

        if len(monthly) < 12:
            raise ValueError(f"Not enough historical data for {commodity} - {market}.")

        # Use last 12 months
        last_12 = monthly[-12:].values

        # Suppress warning
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=UserWarning)
            last_12_scaled = scaler.transform(last_12)

        last_seq = last_12_scaled.reshape(1, 12, 1)

        # Predict next 6 months
        forecast_scaled = model.predict(last_seq, verbose=0)[0]
        forecast = scaler.inverse_transform(forecast_scaled.reshape(-1, 1)).flatten()

        # Build date keys
        last_date = monthly.index[-1]
        future_dates = pd.date_range(start=last_date + pd.DateOffset(months=1), periods=6, freq="ME")
        forecast_dict = {date.strftime("%Y-%m-%d"): round(price, 2) for date, price in zip(future_dates, forecast)}

        return forecast_dict

    except Exception as e:
        print(f"❌ Error in prediction for {commodity}-{market}: {e}")
        return None


In [11]:
df = pd.read_csv("crop_prices.csv")
forecast = predict_next_6_months("Rice", "Sealdah Koley Market", df)

if forecast:
    for date, price in forecast.items():
        print(f"{date}: ₹{price}")



2023-08-31: ₹3037.469970703125
2023-09-30: ₹3034.760009765625
2023-10-31: ₹3021.14990234375
2023-11-30: ₹3034.659912109375
2023-12-31: ₹3157.68994140625
2024-01-31: ₹3062.77001953125
